In [1]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
IMAGE_HEIGHT,IMAGE_WIDTH=64,64#dimensions of the frame
SEQUENCE_LENGTH=20#no owf frames of a video which would be fed
DATASET_DIR=r'C:\Users\ASUS\OneDrive\Desktop\majorproject\hockey'

In [4]:
CLASSES_LIST=["FIGHT","NOFIGHT"]

In [5]:
def frames_extraction(video_path):
    frames_list=[]
    video_reader=cv2.VideoCapture(video_path)
    video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)#the interval after which frrames get aadded
    #iterate thru frames
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
        success,frame=video_reader.read()
        if not success:
            break
        resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
        normalized_frame=resized_frame/255
        frames_list.append(normalized_frame)
    video_reader.release()
    return frames_list

In [6]:
import os
frames2=0
def create_dataset():
    features = []
    labels = []
    video_files_paths = []
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            frames = frames_extraction(video_file_path)
         
            if len(frames) == SEQUENCE_LENGTH:

                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)
    frames2=frames
    
    return features, labels, video_files_paths

In [7]:
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: FIGHT
Extracting Data of Class: NOFIGHT


In [8]:
one_hot_encoded_labels = to_categorical(labels)

In [9]:


features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state=seed_constant
                                                                            )

In [12]:

def create_LRCN_model():
  
   

    model = Sequential()
    
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'tanh'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'tanh')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'tanh')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'tanh')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'sigmoid'))

    model.summary()
    
    return model

In [13]:
LRCN_model = create_LRCN_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 20, 64, 64, 16)    448       
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 20, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 20, 16, 16, 32)    4640      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 20, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 20, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 20, 4, 4, 64)     

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 12, mode = 'min', restore_best_weights = True)

LRCN_model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])


LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 4 ,
                                             shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [ ]:
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)